In [1]:
import cffi
import numba
import numba.core.typing.cffi_utils as cffi_support
import numpy as np

import ufl
from dolfinx.cpp.fem import Form_float64
from dolfinx.jit import ffcx_jit
from dolfinx import fem, mesh, io

from mpi4py import MPI
from petsc4py import PETSc

In [2]:
# Create mesh and function space
domain = mesh.create_unit_square(MPI.COMM_WORLD, 1, 1)
V = fem.FunctionSpace(domain, ("Lagrange", 1))

# Unpack mesh and dofmap data
num_owned_cells = domain.topology.index_map(domain.topology.dim).size_local
num_cells = num_owned_cells + domain.topology.index_map(domain.topology.dim).num_ghosts
x_dofs = domain.geometry.dofmap.array.reshape(num_cells, 3)
x = domain.geometry.x
dofmap = V.dofmap.list.array.reshape(num_cells, 3)

ffcxtype = "double"
nptype = "float64"

ffi = cffi.FFI()
cffi_support.register_type(ffi.typeof("double"), numba.types.float64)
c_signature = numba.types.void(
    numba.types.CPointer(numba.typeof(PETSc.ScalarType())),
    numba.types.CPointer(numba.typeof(PETSc.ScalarType())),
    numba.types.CPointer(numba.typeof(PETSc.ScalarType())),
    numba.types.CPointer(numba.types.double),
    numba.types.CPointer(numba.types.int32),
    numba.types.CPointer(numba.types.uint8))

@numba.njit(fastmath=True)
def assemble_vector_ufc(b, kernel, mesh, dofmap, num_cells):
    """Assemble provided FFCx/UFC kernel over a mesh into the array b"""
    v, x = mesh
    entity_local_index = np.array([0], dtype=np.intc)
    perm = np.array([0], dtype=np.uint8)
    geometry = np.zeros((3, 3))
    coeffs = np.zeros(1, dtype=PETSc.ScalarType)
    constants = np.zeros(1, dtype=PETSc.ScalarType)

    b_local = np.zeros(3, dtype=PETSc.ScalarType)
    for cell in range(num_cells):
        # FIXME: This assumes a particular geometry dof layout
        for j in range(3):
            geometry[j] = x[v[cell, j], :]
        b_local.fill(0.0)
        kernel(ffi.from_buffer(b_local), ffi.from_buffer(coeffs),
               ffi.from_buffer(constants),
               ffi.from_buffer(geometry), ffi.from_buffer(entity_local_index),
               ffi.from_buffer(perm))
        for j in range(3):
            b[dofmap[cell, j]] += b_local[j]

In [3]:
f = 1
const = fem.Constant(domain, PETSc.ScalarType(f))
coeff = fem.Function(fem.FunctionSpace(domain, ('DG', 0)))
coeff.vector.set(f)

In [4]:
# Test against generated code and general assembler
v = ufl.TestFunction(V)
L = ufl.inner(f, v) * ufl.dx
b1 = fem.petsc.assemble_vector(fem.form(L))
print("Correct linear form: ", b1[:])

b3 = fem.Function(V)
b = b3.x.array
b[:] = 0.0
ufcx_form, _, _ = ffcx_jit(domain.comm, L, form_compiler_params={"scalar_type": ffcxtype})
kernel = getattr(ufcx_form.integrals(0)[0], f"tabulate_tensor_{nptype}")
assemble_vector_ufc(b, kernel, (x_dofs, x), dofmap, num_owned_cells)
print('lin form after assemble_vector_ufc with direct constant in the form:', b3.x.array[:])

L = ufl.inner(const, v) * ufl.dx
b4 = fem.Function(V)
b = b4.x.array
b[:] = 0.0
ufcx_form, _, _ = ffcx_jit(domain.comm, L, form_compiler_params={"scalar_type": ffcxtype})
kernel_const = getattr(ufcx_form.integrals(0)[0], f"tabulate_tensor_{nptype}")
assemble_vector_ufc(b, kernel_const, (x_dofs, x), dofmap, num_owned_cells)
print('lin form after assemble_vector_ufc with fem.Constant in the form:', b4.x.array[:])

L = ufl.inner(coeff, v) * ufl.dx
b5 = fem.Function(V)
b = b5.x.array
b[:] = 0.0
ufcx_form, _, _ = ffcx_jit(domain.comm, L, form_compiler_params={"scalar_type": ffcxtype})
kernel_coeff = getattr(ufcx_form.integrals(0)[0], f"tabulate_tensor_{nptype}")
assemble_vector_ufc(b, kernel_coeff, (x_dofs, x), dofmap, num_owned_cells)
print('lin form after assemble_vector_ufc with fem.Constant in the form:', b5.x.array[:])

Correct linear form:  [0.33333333 0.16666667 0.33333333 0.16666667]
lin form after assemble_vector_ufc with direct constant in the form: [0.33333333 0.16666667 0.33333333 0.16666667]
lin form after assemble_vector_ufc with fem.Constant in the form: [0. 0. 0. 0.]
lin form after assemble_vector_ufc with fem.Constant in the form: [0. 0. 0. 0.]


In [5]:
##### cpp/dolfinx/fem/Form.h:

# /// @brief Type of integral
# enum class IntegralType : std::int8_t
# {
#   cell = 0,           ///< Cell
#   exterior_facet = 1, ///< Exterior facet
#   interior_facet = 2, ///< Interior facet
#   vertex = 3          ///< Vertex
# };

# /// @brief A representation of finite element variational forms.
# ///
# /// A note on the order of trial and test spaces: FEniCS numbers
# /// argument spaces starting with the leading dimension of the
# /// corresponding tensor (matrix). In other words, the test space is
# /// numbered 0 and the trial space is numbered 1. However, in order to
# /// have a notation that agrees with most existing finite element
# /// literature, in particular
# ///
# ///  \f[   a = a(u, v)        \f]
# ///
# /// the spaces are numbered from right to left
# ///
# ///  \f[   a: V_1 \times V_0 \rightarrow \mathbb{R}  \f]
# ///
# /// This is reflected in the ordering of the spaces that should be
# /// supplied to generated subclasses. In particular, when a bilinear
# /// form is initialized, it should be initialized as `a(V_1, V_0) =
# /// ...`, where `V_1` is the trial space and `V_0` is the test space.
# /// However, when a form is initialized by a list of argument spaces
# /// (the variable `function_spaces` in the constructors below), the list
# /// of spaces should start with space number 0 (the test space) and then
# /// space number 1 (the trial space).

# template <typename T>
# class Form
# {
# public:
#   /// @brief Create a finite element form.
#   ///
#   /// @note User applications will normally call a fem::Form builder
#   /// function rather using this interfcae directly.
#   ///
#   /// @param[in] function_spaces Function spaces for the form arguments
#   /// @param[in] integrals The integrals in the form. The first key is
#   /// the domain type. For each key there is a pair (list[domain id,
#   /// integration kernel], domain markers).
#   /// @param[in] coefficients
#   /// @param[in] constants Constants in the Form
#   /// @param[in] needs_facet_permutations Set to true is any of the
#   /// integration kernels require cell permutation data
#   /// @param[in] mesh The mesh of the domain. This is required when
#   /// there are not argument functions from which the mesh can be
#   /// extracted, e.g. for functionals

In [6]:
@numba.cfunc(c_signature, nopython=True)
def tabulate_lin_form_b(b_, w_, c_, coords_, local_index, orientation):
    b = numba.carray(b_, (3), dtype=PETSc.ScalarType)      
    b_tmp = np.zeros((3), dtype=PETSc.ScalarType)
    w = numba.carray(w_, (1), dtype=PETSc.ScalarType)
    w[0] = 1 # Changing of the coefficient value on an element !
    kernel_coeff(ffi.from_buffer(b_tmp), w_, c_, coords_, local_index, orientation)
    b[:] = b_tmp

    # It works the same way:
    # 1) kernel_coeff(b_, w_, c_, coords_, local_index, orientation)
    # 2) b = numba.carray(b_, (3), dtype=PETSc.ScalarType)      
    #    kernel_coeff(ffi.from_buffer(b), w_, c_, coords_, local_index, orientation)

integrals = {fem.IntegralType.cell: ([(-1, tabulate_lin_form_b.address)], None)}
L = Form_float64([V._cpp_object], integrals, [coeff._cpp_object], [], False)
b = fem.petsc.assemble_vector(L)
print('lin form after any sort of tabulation', b[:])

@numba.cfunc(c_signature, nopython=True)
def tabulate_lin_form_b_coeff(b_, w_, c_, coords_, local_index, orientation=ffi.NULL):
    b = numba.carray(b_, (3), dtype=PETSc.ScalarType)
    w = numba.carray(w_, (1), dtype=PETSc.ScalarType)
    c = numba.carray(c_, (1), dtype=PETSc.ScalarType)
    coordinate_dofs = numba.carray(coords_, (3, 3), dtype=np.float64)
    x0, y0 = coordinate_dofs[0, :2]
    x1, y1 = coordinate_dofs[1, :2]
    x2, y2 = coordinate_dofs[2, :2]

    # 2x Element area Ae
    Ae = abs((x0 - x1) * (y2 - y1) - (y0 - y1) * (x2 - x1))
    b[:] = c[0] * Ae / 6.0

integrals = {fem.IntegralType.cell: ([(-1, tabulate_lin_form_b_coeff.address)], None)}
L = Form_float64([V._cpp_object], integrals, [coeff._cpp_object], [const._cpp_object], False)
b = fem.petsc.assemble_vector(L)
print('lin form after direct calculation', b[:])

lin form after any sort of tabulation [0.33333333 0.16666667 0.33333333 0.16666667]
lin form after direct calculation [0.33333333 0.16666667 0.33333333 0.16666667]


In [7]:
u, v = ufl.TrialFunction(V), ufl.TestFunction(V)
a = ufl.inner( f * u, v ) * ufl.dx
A = fem.petsc.assemble_matrix(fem.form(a))
A.assemble()
print('Correct bil form \n', A[:, :])

Correct bil form 
 [[0.16666667 0.04166667 0.08333333 0.04166667]
 [0.04166667 0.08333333 0.04166667 0.        ]
 [0.08333333 0.04166667 0.16666667 0.04166667]
 [0.04166667 0.         0.04166667 0.08333333]]


In [8]:
ufcx_form, _, _ = ffcx_jit(domain.comm, a, form_compiler_params={"scalar_type": ffcxtype})
kernel_a = getattr(ufcx_form.integrals(0)[0], f"tabulate_tensor_{nptype}")

@numba.cfunc(c_signature, nopython=True)
def tabulate_tensor_A(A_, w_, c_, coords_, entity_local_index, permutation):
    # A = numba.carray(A_, (3, 3), dtype=PETSc.ScalarType)
    kernel_a(A_, w_, c_, coords_, entity_local_index, permutation)


integrals = {fem.IntegralType.cell: ([(-1, tabulate_tensor_A.address)], None)}
a_form = Form_float64([V._cpp_object, V._cpp_object], integrals, [], [], False, None)
A = fem.petsc.assemble_matrix(a_form)
A.assemble()
print(A[:, :])  

[[0.16666667 0.04166667 0.08333333 0.04166667]
 [0.04166667 0.08333333 0.04166667 0.        ]
 [0.08333333 0.04166667 0.16666667 0.04166667]
 [0.04166667 0.         0.04166667 0.08333333]]


In [9]:
a = ufl.inner( const * u, v ) * ufl.dx

ufcx_form, _, _ = ffcx_jit(domain.comm, a, form_compiler_params={"scalar_type": ffcxtype})
kernel_a = getattr(ufcx_form.integrals(0)[0], f"tabulate_tensor_{nptype}")

@numba.cfunc(c_signature, nopython=True)
def tabulate_tensor_A(A_, w_, c_, coords_, entity_local_index, permutation):
    # A = numba.carray(A_, (3, 3), dtype=PETSc.ScalarType)
    kernel_a(A_, w_, c_, coords_, entity_local_index, permutation)

integrals = {fem.IntegralType.cell: ([(-1, tabulate_tensor_A.address)], None)}
a_form = Form_float64([V._cpp_object, V._cpp_object], integrals, [], [const._cpp_object], False, None)
A = fem.petsc.assemble_matrix(a_form)
A.assemble()
print(A[:, :])  

[[0.16666667 0.04166667 0.08333333 0.04166667]
 [0.04166667 0.08333333 0.04166667 0.        ]
 [0.08333333 0.04166667 0.16666667 0.04166667]
 [0.04166667 0.         0.04166667 0.08333333]]


In [10]:
a = ufl.inner( const * u, v ) * ufl.dx

ufcx_form, _, _ = ffcx_jit(domain.comm, a, form_compiler_params={"scalar_type": ffcxtype})
kernel_a = getattr(ufcx_form.integrals(0)[0], f"tabulate_tensor_{nptype}")

@numba.cfunc(c_signature, nopython=True)
def tabulate_tensor_A(A_, w_, c_, coords_, entity_local_index, permutation):
    # A = numba.carray(A_, (3, 3), dtype=PETSc.ScalarType)
    c_[0] = 2 # !!!!!
    kernel_a(A_, w_, c_, coords_, entity_local_index, permutation)

integrals = {fem.IntegralType.cell: ([(-1, tabulate_tensor_A.address)], None)}
a_form = Form_float64([V._cpp_object, V._cpp_object], integrals, [], [const._cpp_object], False, None)
A = fem.petsc.assemble_matrix(a_form)
A.assemble()
print(A[:, :])  

[[0.33333333 0.08333333 0.16666667 0.08333333]
 [0.08333333 0.16666667 0.08333333 0.        ]
 [0.16666667 0.08333333 0.33333333 0.08333333]
 [0.08333333 0.         0.08333333 0.16666667]]


In [11]:
a = ufl.inner( coeff * u, v ) * ufl.dx

ufcx_form, _, _ = ffcx_jit(domain.comm, a, form_compiler_params={"scalar_type": ffcxtype})
kernel_a = getattr(ufcx_form.integrals(0)[0], f"tabulate_tensor_{nptype}")

@numba.cfunc(c_signature, nopython=True)
def tabulate_tensor_A(A_, w_, c_, coords_, entity_local_index, permutation):
    w_[0] = 2 # !!!!!
    kernel_a(A_, w_, c_, coords_, entity_local_index, permutation)

integrals = {fem.IntegralType.cell: ([(-1, tabulate_tensor_A.address)], None)}
a_form = Form_float64([V._cpp_object, V._cpp_object], integrals, [coeff._cpp_object], [const._cpp_object], False, None)
A = fem.petsc.assemble_matrix(a_form)
A.assemble()
print(A[:, :])  

[[0.33333333 0.08333333 0.16666667 0.08333333]
 [0.08333333 0.16666667 0.08333333 0.        ]
 [0.16666667 0.08333333 0.33333333 0.08333333]
 [0.08333333 0.         0.08333333 0.16666667]]


In [12]:
V = fem.VectorFunctionSpace(domain, ("Lagrange", 1))
u, v = ufl.TrialFunction(V), ufl.TestFunction(V)

const = fem.Constant(domain, (PETSc.ScalarType(1), PETSc.ScalarType(1)))

L = ufl.inner(const, v) * ufl.dx
b1 = fem.petsc.assemble_vector(fem.form(L))
print("Correct linear form:\n", b1[:])

# coeff = fem.Function(fem.FunctionSpace(domain, ('DG', 0)))
# coeff.vector.set(f)

# a = ufl.inner( f * u, v ) * ufl.dx
# A = fem.petsc.assemble_matrix(fem.form(a))
# A.assemble()
# print('Correct bil form \n', A[:, :])

Correct linear form:
 [0.33333333 0.33333333 0.16666667 0.16666667 0.33333333 0.33333333
 0.16666667 0.16666667]


In [13]:
L = ufl.inner(const, v) * ufl.dx

ufcx_form, _, _ = ffcx_jit(domain.comm, L, form_compiler_params={"scalar_type": ffcxtype})
kernel_b = getattr(ufcx_form.integrals(0)[0], f"tabulate_tensor_{nptype}")

@numba.cfunc(c_signature, nopython=True)
def tabulate_lin_form_b(b_, w_, c_, coords_, local_index, orientation):
    c = numba.carray(c_, (2), dtype=PETSc.ScalarType)
    c[0] = 0 # Changing of the coefficient value on an element !
    kernel_b(b_, w_, c_, coords_, local_index, orientation)
    
integrals = {fem.IntegralType.cell: ([(-1, tabulate_lin_form_b.address)], None)}
L = Form_float64([V._cpp_object], integrals, [], [const._cpp_object], False)
b = fem.petsc.assemble_vector(L)
print(b[:])

[0.         0.33333333 0.         0.16666667 0.         0.33333333
 0.         0.16666667]


In [14]:
a = ufl.inner( coeff * u, v ) * ufl.dx
A = fem.petsc.assemble_matrix(fem.form(a))
A.assemble()
print(A[:, :])

[[0.16666667 0.         0.04166667 0.         0.08333333 0.
  0.04166667 0.        ]
 [0.         0.16666667 0.         0.04166667 0.         0.08333333
  0.         0.04166667]
 [0.04166667 0.         0.08333333 0.         0.04166667 0.
  0.         0.        ]
 [0.         0.04166667 0.         0.08333333 0.         0.04166667
  0.         0.        ]
 [0.08333333 0.         0.04166667 0.         0.16666667 0.
  0.04166667 0.        ]
 [0.         0.08333333 0.         0.04166667 0.         0.16666667
  0.         0.04166667]
 [0.04166667 0.         0.         0.         0.04166667 0.
  0.08333333 0.        ]
 [0.         0.04166667 0.         0.         0.         0.04166667
  0.         0.08333333]]


In [15]:
coeff2 = fem.Function(fem.FunctionSpace(domain, ('DG', 0)))
coeff2.vector.set(5)

a = ufl.inner( coeff * u, v * coeff2) * ufl.dx

ufcx_form, _, _ = ffcx_jit(domain.comm, a, form_compiler_params={"scalar_type": ffcxtype})
kernel_a = getattr(ufcx_form.integrals(0)[0], f"tabulate_tensor_{nptype}")

@numba.cfunc(c_signature, nopython=True)
def tabulate_tensor_A(A_, w_, c_, coords_, entity_local_index, permutation):
    kernel_a(A_, w_, c_, coords_, entity_local_index, permutation)

integrals = {fem.IntegralType.cell: ([(-1, tabulate_tensor_A.address)], None)}
a_form = Form_float64([V._cpp_object, V._cpp_object], integrals, [coeff._cpp_object, coeff2._cpp_object], [const._cpp_object], False, None)
A = fem.petsc.assemble_matrix(a_form)
A.assemble()
print(A[:, :])  

[[0.83333333 0.         0.20833333 0.         0.41666667 0.
  0.20833333 0.        ]
 [0.         0.83333333 0.         0.20833333 0.         0.41666667
  0.         0.20833333]
 [0.20833333 0.         0.41666667 0.         0.20833333 0.
  0.         0.        ]
 [0.         0.20833333 0.         0.41666667 0.         0.20833333
  0.         0.        ]
 [0.41666667 0.         0.20833333 0.         0.83333333 0.
  0.20833333 0.        ]
 [0.         0.41666667 0.         0.20833333 0.         0.83333333
  0.         0.20833333]
 [0.20833333 0.         0.         0.         0.20833333 0.
  0.41666667 0.        ]
 [0.         0.20833333 0.         0.         0.         0.20833333
  0.         0.41666667]]


In [16]:
def epsilon(u):
    return ufl.sym(ufl.grad(u))

def sigma(u):
    return 1*ufl.div(u)*ufl.Identity(u.geometric_dimension()) + 2*1*epsilon(u)

a = ufl.inner( sigma(u), epsilon(v) ) * ufl.dx

A = fem.petsc.assemble_matrix(fem.form(a))
A.assemble()
print(A[:, :])

[[ 2.   0.  -1.5  0.5  0.  -1.  -0.5  0.5]
 [ 0.   2.   0.5 -0.5 -1.   0.   0.5 -1.5]
 [-1.5  0.5  2.  -1.  -0.5  0.5  0.   0. ]
 [ 0.5 -0.5 -1.   2.   0.5 -1.5  0.   0. ]
 [ 0.  -1.  -0.5  0.5  2.   0.  -1.5  0.5]
 [-1.   0.   0.5 -1.5  0.   2.   0.5 -0.5]
 [-0.5  0.5  0.   0.  -1.5  0.5  2.  -1. ]
 [ 0.5 -1.5  0.   0.   0.5 -0.5 -1.   2. ]]


In [32]:
coeff1 = fem.Function(fem.FunctionSpace(domain, ('DG', 0)))
coeff2 = fem.Function(fem.FunctionSpace(domain, ('DG', 0)))
coeff1.vector.set(1)
coeff2.vector.set(1)

def epsilon(u):
    return ufl.sym(ufl.grad(u))

def sigma(u):
    return coeff1*ufl.div(u)*ufl.Identity(u.geometric_dimension()) + 2*coeff2*epsilon(u)

a = ufl.inner( sigma(u), epsilon(v) ) * ufl.dx

ufcx_form, _, _ = ffcx_jit(domain.comm, a, form_compiler_params={"scalar_type": ffcxtype})
kernel_a = getattr(ufcx_form.integrals(0)[0], f"tabulate_tensor_{nptype}")

@numba.cfunc(c_signature, nopython=True)
def tabulate_tensor_A(A_, w_, c_, coords_, entity_local_index, permutation):
    w_[0] = 10
    w_[1] = 1000
    kernel_a(A_, w_, c_, coords_, entity_local_index, permutation)

integrals = {fem.IntegralType.cell: ([(-1, tabulate_tensor_A.address)], None)}
a_form = Form_float64([V._cpp_object, V._cpp_object], integrals, [coeff1._cpp_object, coeff2._cpp_object], [const._cpp_object], False, None)
A = fem.petsc.assemble_matrix(a_form)
A.assemble()
print(A[:, :])  

[[ 1505.     0. -1005.     5.     0.  -505.  -500.   500.]
 [    0.  1505.   500.  -500.  -505.     0.     5. -1005.]
 [-1005.   500.  1505.  -505.  -500.     5.     0.     0.]
 [    5.  -500.  -505.  1505.   500. -1005.     0.     0.]
 [    0.  -505.  -500.   500.  1505.     0. -1005.     5.]
 [ -505.     0.     5. -1005.     0.  1505.   500.  -500.]
 [ -500.     5.     0.     0. -1005.   500.  1505.  -505.]
 [  500. -1005.     0.     0.     5.  -500.  -505.  1505.]]
